# 5. Neural Network

**I know this is beyond the scope of the course, but I've been learning about neural networks out of interest lately and I wanted to try building one to test myself on this subject. Before I started, I realized that this model would probably perform worse than the more fundamental methods, but that was not the goal. The goal was to learn something and have fun, which I most certainly did.**

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2022-01-25 08:48:08.662186: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-25 08:48:08.662263: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Some minor errors and warnings, nothing to worry about.

**Let's first build a model using Keras library. I have a laptop without a gpu, so my neural network will have to train on my cpu. Also I don't have many days left to train my model so I will definetely have to make some sacrifices on performance.**

**First of all I made the choice to use only 3 layers (1 hidden layer), otherwise training would take much to long.**

**For my activation functions on the first 2 layers, I chose for ReLU, as it is a fast yet quite reliable choice for regression. as my third layer only has 1 neuron with a single value, I did not need an activation function for that.**

**With my optimizer and loss functions, I didn't go to fancy either, I mainly needed a fast and simple functions. Furthermore, mse is often a good choice for 'simple' regression tasks.**

In [40]:
def build_model():
    model = keras.Sequential([
        layers.Dense(64, activation=tf.nn.relu, input_shape=[len(train.keys())]),
        layers.Dense(64, activation=tf.nn.relu),
        layers.Dense(1)
    ])
    
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    
    model.compile(loss='mse',
                 optimizer=optimizer,
                 metrics=['mae', 'mse'])
    return model

In [2]:
model = build_model()

NameError: name 'build_model' is not defined

**(I accidentally tried running this cell again after training my model but I didn't want to run everything over again so that it why it now says build_model not found, before this worked ofcourse.)**

**Let's have a look at our model now**

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                384       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4,609
Trainable params: 4,609
Non-trainable params: 0
_________________________________________________________________


**Looks good, now let's test if our model is set up correctly by testing it on a small subset of the data.**

In [43]:
example_batch = train[:10]
example_result = model.predict(example_batch)
example_result

array([[-0.05787908],
       [ 0.18334435],
       [ 0.37936437],
       [ 0.2206813 ],
       [ 0.10483198],
       [-0.07229923],
       [-0.0250116 ],
       [ 0.1843274 ],
       [ 0.08441111],
       [ 0.20493421]], dtype=float32)

**Everything seems to work fine, let's train the model now.**

**As this will take hours, I used a printDot() function to print a dot on every iteration of training, to make sure the training is still busy.**

**Then I started training the model with 1000 epochs**

In [44]:
#train the model
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: pr 
              int('')
        print('.', end='')
            
EPOCHS = 1000

history = model.fit(train, train_labels, epochs=EPOCHS, validation_split = 0.2, verbose = 0, callbacks=[PrintDot()])


....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
..........................................................................................

**Done, this took about 3 hours to train already, so unfortunately I won't have the time to tweak a lot with my loss-, activation-, and optimizer functions. Let's hold on to the basics.**

**Let's now check the results of our last training iterations.**

In [45]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,mae,mse,val_loss,val_mae,val_mse,epoch
995,15706116.0,2616.107178,15706116.0,16305415.0,2645.844971,16305415.0,995
996,15697392.0,2615.675049,15697392.0,16330255.0,2640.858887,16330255.0,996
997,15704668.0,2615.661133,15704668.0,16375983.0,2636.772461,16375983.0,997
998,15655734.0,2613.237061,15655734.0,16355840.0,2653.677979,16355840.0,998
999,15655700.0,2614.472900,15655700.0,16400591.0,2652.961670,16400591.0,999


**To compare these results with our other predictive models, I decided to calcultate the rmse from the mse column, by taking the square root.**

In [46]:
hist['rmse'] = hist['mse']**(1/2)
hist.head(1000)

,loss,mae,mse,val_loss,val_mae,val_mse,epoch,rmse
0,252111808.0,12809.167969,252111808.0,43941684.0,4920.054688,43941684.0,0,15878.029097
1,33763576.0,4027.907959,33763576.0,26813236.0,3530.665283,26813236.0,1,5810.643338
2,25565760.0,3350.553711,25565760.0,23330678.0,3242.599609,23330678.0,2,5056.259487
3,23724400.0,3195.875732,23724400.0,22420352.0,3152.117676,22420352.0,3,4870.769960
4,23093154.0,3147.664795,23093154.0,22037216.0,3134.419189,22037216.0,4,4805.533685
...,...,...,...,...,...,...,...,...
995,15706116.0,2616.107178,15706116.0,16305415.0,2645.844971,16305415.0,995,3963.094246
996,15697392.0,2615.675049,15697392.0,16330255.0,2640.858887,16330255.0,996,3961.993438
997,15704668.0,2615.661133,15704668.0,16375983.0,2636.772461,16375983.0,997,3962.911556
998,15655734.0,2613.237061,15655734.0,16355840.0,2653.677979,16355840.0,998,3956.732743


**Not great, but understandable with only one hidden layer.**

**Now let's use the EarlyStopping()function from Keras library to automatically find out at how many epochs the model doesn't improve anymore and can stop.**

In [47]:
early_stop = keras.callbacks.EarlyStopping(monitor='mse', patience=20)

EPOCHS = 10000
history = model.fit(train, train_labels, epochs=EPOCHS, 
                    validation_split=0.2, verbose=0, callbacks=[early_stop, PrintDot()])



.....................

In [48]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist['rmse'] = hist['mse']**(1/2)
hist.tail()

,loss,mae,mse,val_loss,val_mae,val_mse,epoch,rmse
16,15711818.0,2614.414307,15711818.0,16939938.0,2730.013184,16939938.0,16,3963.813568
17,15723955.0,2615.282959,15723955.0,16436625.0,2639.060059,16436625.0,17,3965.344247
18,15701191.0,2613.296631,15701191.0,17102832.0,2737.846436,17102832.0,18,3962.472839
19,15690877.0,2615.295410,15690877.0,16380891.0,2633.541992,16380891.0,19,3961.171165
20,15656747.0,2613.933350,15656747.0,16787302.0,2642.258301,16787302.0,20,3956.860751


**Only 20 epochs now, that's not so much. Impossible to train a neural network with only 20 iterations, tweaking some variables and monitoring another function instead of 'mse' would probably help here, but unfortunately time is limited. Maybe one day I'll optimize this neural network, but for now I'll call it a day.**


Please note that while making this project by myself, I sometimes used (parts) of lines of code from other sources. I do not own these code snippets and I would hereby like to list my code and learning sources:


-Hands-On machine learning with Scikil-Learn, Keras & TensorFlow book and pdf version

-Official wiki pages from all libraries used

-Stackoverflow.com

-Youtube.com